In [35]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 18: RAM Run ---</h2><p>You and The Historians look a lot more pixelated than you remember. You're <a href="/2017/day/2">inside a computer</a> at the North Pole!</p>
<p>Just as you're about to check out your surroundings, a program runs up to you. "This region of memory isn't safe! The User misunderstood what a <a href="https://en.wikipedia.org/wiki/Pushdown_automaton" target="_blank">pushdown automaton</a> is and their algorithm is pushing whole <em>bytes</em> down on top of us! <span title="Pun intended.">Run</span>!"</p>
<p>The algorithm is fast - it's going to cause a byte to fall into your memory space once every <a href="https://www.youtube.com/watch?v=9eyFDBPk4Yw" target="_blank">nanosecond</a>! Fortunately, you're <em>faster</em>, and by quickly scanning the algorithm, you create a <em>list of which bytes will fall</em> (your puzzle input) in the order they'll land in your memory space.</p>
<p>Your memory space is a two-dimensional grid with coordinates that range from <code>0</code> to <code>70</code> both horizontally and vertically. However, for the sake of example, suppose you're on a smaller grid with coordinates that range from <code>0</code> to <code>6</code> and the following list of incoming byte positions:</p>
<pre><code>5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
</code></pre>
<p>Each byte position is given as an <code>X,Y</code> coordinate, where <code>X</code> is the distance from the left edge of your memory space and <code>Y</code> is the distance from the top edge of your memory space.</p>
<p>You and The Historians are currently in the top left corner of the memory space (at <code>0,0</code>) and need to reach the exit in the bottom right corner (at <code>70,70</code> in your memory space, but at <code>6,6</code> in this example). You'll need to simulate the falling bytes to plan out where it will be safe to run; for now, simulate just the first few bytes falling into your memory space.</p>
<p>As bytes fall into your memory space, they make that coordinate <em>corrupted</em>. Corrupted memory coordinates cannot be entered by you or The Historians, so you'll need to plan your route carefully. You also cannot leave the boundaries of the memory space; your only hope is to reach the exit.</p>
<p>In the above example, if you were to draw the memory space after the first <code>12</code> bytes have fallen (using <code>.</code> for safe and <code>#</code> for corrupted), it would look like this:</p>
<pre><code>...#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#....
</code></pre>
<p>You can take steps up, down, left, or right. After just 12 bytes have corrupted locations in your memory space, the shortest path from the top left corner to the exit would take <code><em>22</em></code> steps. Here (marked with <code>O</code>) is one such path:</p>
<pre><code><em>O</em><em>O</em>.#<em>O</em><em>O</em><em>O</em>
.<em>O</em>#<em>O</em><em>O</em>#<em>O</em>
.<em>O</em><em>O</em><em>O</em>#<em>O</em><em>O</em>
...#<em>O</em><em>O</em>#
..#<em>O</em><em>O</em>#.
.#.<em>O</em>#..
#.#<em>O</em><em>O</em><em>O</em><em>O</em>
</code></pre>
<p>Simulate the first kilobyte (<code>1024</code> bytes) falling onto your memory space. Afterward, <em>what is the minimum number of steps needed to reach the exit?</em></p>
</article>


In [36]:
from collections import deque
from itertools import islice


tests = [
    {
        "name": "Example",
        "s": """
            5,4
            4,2
            4,5
            3,0
            2,1
            6,3
            2,4
            1,5
            0,6
            3,3
            2,6
            5,1
            1,2
            5,5
            2,5
            6,5
            1,4
            0,4
            6,4
            1,1
            6,1
            1,0
            0,5
            1,6
            2,0
        """,
        "first_n_bytes": 12,
        "expected": 22,
    },
]


class Memory(Str):
    DIRECTIONS = (-1, 0), (0, 1), (1, 0), (0, -1)

    def __init__(self, s: str, right_bottom: int) -> None:
        self.right_bottom = right_bottom
        self.cols, self.rows = (i + 1 for i in right_bottom)
        self.memory = [["."] * self.cols for _ in range(self.rows)]
        self.bytes = [
            tuple(map(int, re.findall(r"\d+", row))) for row in s.strip().splitlines()
        ]

    def min_steps_to_goal(self) -> int:
        queue = deque([(0, 0)])
        seen = set()
        steps = 0

        while queue:
            for _ in range(len(queue)):
                row, col = queue.popleft()

                if (row, col) == self.right_bottom:
                    return steps

                if not (0 <= row < self.rows and 0 <= col < self.cols):
                    continue

                if self.memory[row][col] == "#":
                    continue

                if (row, col) in seen:
                    continue

                seen.add((row, col))

                queue.extend((row + dr, col + dc) for dr, dc in self.DIRECTIONS)

            steps += 1

        return -1

    def _place_first_n_bytes(self, n: int) -> None:
        for col, row in islice(self.bytes, n):
            self.memory[row][col] = "#"

    def __str__(self) -> str:
        return "\n".join("".join(row) for row in self.memory)

    @classmethod
    def partI(cls, s: str, first_n_bytes: int, right_bottom=(6, 6)) -> int:
        m = cls(s, right_bottom)
        m._place_first_n_bytes(first_n_bytes)
        result = m.min_steps_to_goal()
        return result


@test(tests=tests[:])
def partI_test(s: str, first_n_bytes: int, right_bottom=(6, 6)) -> int:
    return Memory.partI(s, first_n_bytes, right_bottom)


Test Example passed, for partI_test.
Success


In [37]:
with open("../input/day18.txt") as f:
    puzzle = f.read()

print(f"Part I: {Memory.partI(puzzle, first_n_bytes=1024, right_bottom=(70,70))}")

Part I: 326


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>326</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<art<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Historians aren't as used to moving around in this pixelated universe as you are. You're afraid they're not going to be fast enough to make it to the exit before the path is completely blocked.</p>
<p>To determine how fast everyone needs to go, you need to determine <em>the first byte that will cut off the path to the exit</em>.</p>
<p>In the above example, after the byte at <code>1,1</code> falls, there is still a path to the exit:</p>
<pre><code><em>O</em>..#<em>O</em><em>O</em><em>O</em>
<em>O</em>##<em>O</em><em>O</em>#<em>O</em>
<em>O</em>#<em>O</em><em>O</em>#<em>O</em><em>O</em>
<em>O</em><em>O</em><em>O</em>#<em>O</em><em>O</em>#
###<em>O</em><em>O</em>##
.##<em>O</em>###
#.#<em>O</em><em>O</em><em>O</em><em>O</em>
</code></pre>
<p>However, after adding the very next byte (at <code>6,1</code>), there is no longer a path to the exit:</p>
<pre><code>...#...
.##..#<em>#</em>
.#..#..
...#..#
###..##
.##.###
#.#....
</code></pre>
<p>So, in this example, the coordinates of the first byte that prevents the exit from being reachable are <code><em>6,1</em></code>.</p>
<p>Simulate more of the bytes that are about to corrupt your memory space. <em>What are the coordinates of the first byte that will prevent the exit from being reachable from your starting position?</em> (Provide the answer as two integers separated by a comma with no other characters.)</p>
</article>


In [41]:
tests = [
    {
        "name": "Example",
        "s": """
            5,4
            4,2
            4,5
            3,0
            2,1
            6,3
            2,4
            1,5
            0,6
            3,3
            2,6
            5,1
            1,2
            5,5
            2,5
            6,5
            1,4
            0,4
            6,4
            1,1
            6,1
            1,0
            0,5
            1,6
            2,0
        """,
        "expected": (6, 1),
    },
]


class MemoryII(Memory):
    def place_byte_fr_to(self, fr: int, to: int, value: chr = "#") -> None:
        for i in range(fr, to):
            c, r = self.bytes[i]
            self.memory[r][c] = value

    def remove_byte_fr_to(self, fr: int, to: int) -> None:
        self.place_byte_fr_to(fr, to, ".")

    def first_blocking_byte(self) -> tuple[int, int]:
        fr, to = 0, len(self.bytes)

        while fr < to:
            mid = (fr + to) // 2
            self.place_byte_fr_to(fr, mid)

            if to - fr == 1:
                return self.bytes[fr]

            if self.min_steps_to_goal() == -1:
                self.remove_byte_fr_to(fr, mid)
                to = mid
            else:
                fr = mid

        return -1

    @classmethod
    def partII(cls, s: str, right_bottom) -> tuple[int, int]:
        m = cls(s, right_bottom)
        return m.first_blocking_byte()


@test(tests=tests[:])
def partI_test(s: str, right_bottom=(6, 6)) -> int:
    return MemoryII.partII(s, right_bottom=right_bottom)


Test Example passed, for partI_test.
Success


In [42]:
print(f"Part II: {MemoryII.partII(puzzle,  right_bottom=(70,70))}")

Part II: (18, 62)


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>18,62</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
